In [122]:
# Importing libraries...
import pandas as pd
import matplotlib.pyplot as plt
!matplotlib inline
import seaborn as sns; sns.set()
import zipfile
from bs4 import BeautifulSoup
import os
import numpy as np

'matplotlib' is not recognized as an internal or external command,
operable program or batch file.


In [123]:
df_bestofrt = pd.read_csv('data/bestofrt.tsv', sep='\t')

In [124]:
df_bestofrt.head()

,ranking,critic_score,title,number_of_critic_ratings
0,1,99,The Wizard of Oz (1939),110
1,2,100,Citizen Kane (1941),75
2,3,100,The Third Man (1949),77
3,4,99,Get Out (2017),282
4,5,97,Mad Max: Fury Road (2015),370


In [153]:
with zipfile.ZipFile('data/rt-html.zip', 'r') as myzip:
    myzip.extractall(path='data/')

In [126]:
# List of dictionaries to build file by file and later convert to a DataFrame
df_list = []
folder = 'data/rt_html'

for movie_html in os.listdir(folder):
    with open(os.path.join(folder, movie_html)) as file:
        soup = BeautifulSoup(file, 'lxml')
        title = soup.find('title').contents[0][:-len(' - Rotten Tomatoes')]
        audience_score = soup.find('div', class_='audience-score meter').find('span').contents[0][:-1]
        num_audience_ratings = soup.find('div', class_='audience-info hidden-xs superPageFontColor')
        num_audience_ratings = num_audience_ratings.find_all('div')[1].contents[2].strip().replace(',', '')
        
        # Append to list of dictionaries
        df_list.append({'title': title,
                        'audience_score': int(audience_score),
                        'number_of_audience_ratings': int(num_audience_ratings)})
df = pd.DataFrame(df_list, columns = ['title', 'audience_score', 'number_of_audience_ratings'])

In [127]:
df.head()

,title,audience_score,number_of_audience_ratings
0,12 Angry Men (Twelve Angry Men) (1957),97,103672
1,The 39 Steps (1935),86,23647
2,The Adventures of Robin Hood (1938),89,33584
3,All About Eve (1950),94,44564
4,All Quiet on the Western Front (1930),89,17768


In [128]:
df['title'] = df['title'].str.replace(u'\xa0', ' ')

In [129]:
df.sort_values('title', inplace=True, ignore_index=True)

In [130]:
df_bestofrt.sort_values('title', inplace=True, ignore_index=True)

In [131]:
unmatched_from_html = df[df['title'] != df_bestofrt['title']]['title'].values.reshape(3, 1)
unmatched_from_html

array([["Army of Shadows (L'ArmÃ©e des ombres) (1969)"],
       ['RashÃ´mon (1951)'],
       ['Tokyo Story (TÃ´kyÃ´ monogatari) (1953)']], dtype=object)

In [132]:
unmatched_from_csv = df_bestofrt[df_bestofrt['title'] != df['title']]['title'].values.reshape(3, 1)
unmatched_from_csv

array([["Army of Shadows (L'Armée des ombres) (1969)"],
       ['Rashômon (1951)'],
       ['Tokyo Story (Tôkyô monogatari) (1953)']], dtype=object)

In [134]:
merged = np.concatenate((unmatched_from_html, unmatched_from_csv), axis=1)
merged

array([["Army of Shadows (L'ArmÃ©e des ombres) (1969)",
        "Army of Shadows (L'Armée des ombres) (1969)"],
       ['RashÃ´mon (1951)', 'Rashômon (1951)'],
       ['Tokyo Story (TÃ´kyÃ´ monogatari) (1953)',
        'Tokyo Story (Tôkyô monogatari) (1953)']], dtype=object)

In [141]:
merged[0,0]

"Army of Shadows (L'ArmÃ©e des ombres) (1969)"

In [146]:
for i in range(3):
    df.loc[df.title == merged[i, 0], "title"] = merged[i, 1]

In [150]:
df_combined = df.merge(df_bestofrt, on='title', how='inner')

In [151]:
df_combined

,title,audience_score,number_of_audience_ratings,ranking,critic_score,number_of_critic_ratings
0,12 Angry Men (Twelve Angry Men) (1957),97,103672,53,100,49
1,12 Years a Slave (2013),90,138789,29,96,316
2,A Hard Day's Night (1964),89,50067,22,98,104
3,A Streetcar Named Desire (1951),90,54761,60,98,54
4,Alien (1979),94,457186,48,97,104
...,...,...,...,...,...,...
95,Toy Story 3 (2010),89,605098,39,99,291
96,Up (2009),90,1201878,52,98,286
97,Vertigo (1958),93,101454,66,97,64
98,Wonder Woman (2017),90,112955,46,92,333


In [152]:
df_combined.to_csv('data/combined.csv')